In [ ]:
import requests
import json
import base64

# 1) Define the endpoint URL for your FastAPI app
url = "http://localhost:8000/calculate_plate"

# 2) Prepare job_data payload
job_data = {
    "_meta": {"job_id": "_00000000001"},
    "geometry": {
        "width": 10.0,
        "height": 10.0,
        "thickness": 0.1
    },
    "material": {
        "youngs_modulus": 210e9,
        "poissons_ratio": 0.3,
        "yield_strength": 250e6
    },
    "calculation": {
        "nx": 10,
        "ny": 10
    },
    "load": {
        "position": "[5, 5, 0]",
        "value": "[0, 0, -1000]"
    },
    "output": {}
}

# 3) Send the POST request
response = requests.post(url, json=job_data)

# 4) Check for errors
if response.status_code != 200:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)
else:
    # Parse the JSON response
    resp_data = response.json()

    # Print out some info about the response
    print("Metadata (updated job_data):")
    print(json.dumps(resp_data["metadata"], indent=2))

    # Extract base64-encoded parquet files
    res2d_b64 = resp_data["parquet_files"]["res2d"]["base64_data"]
    res3d_b64 = resp_data["parquet_files"]["res3d"]["base64_data"]

    # Decode and save them locally
    res2d_binary = base64.b64decode(res2d_b64)
    with open("local_res2d.parquet", "wb") as f:
        f.write(res2d_binary)

    res3d_binary = base64.b64decode(res3d_b64)
    with open("local_res3d.parquet", "wb") as f:
        f.write(res3d_binary)

    print("\nFiles 'local_res2d.parquet' and 'local_res3d.parquet' have been saved locally.")
